In [1]:
import pandas as pd
from sklearn.metrics import classification_report
from transformers import AutoTokenizer, AutoModelForCausalLM, pipeline,AutoModelForSeq2SeqLM

import torch
print(torch.__version__)
print(torch.cuda.is_available())
from transformers.utils import is_torch_available, is_tf_available

print("Torch disponible:", is_torch_available())
print("TensorFlow disponible:", is_tf_available())



c:\Users\migue\Documents\GitHub\Practica_BAIN_4\.env\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


2.5.1+cu121
True
Torch disponible: True
TensorFlow disponible: False


In [2]:
df = pd.read_csv('INPUT/emociones.csv')
df = df[['opinion', 'label']]
df = df.dropna()
df = df.sample(30, random_state=42).reset_index(drop=True)
print(df)


                       opinion     label
0          Es un gran servicio  positiva
1                   Es regular   neutral
2          Es un producto útil  positiva
3                 Es un fraude  negativa
4               Me gusta mucho  positiva
5               Es un desastre  negativa
6           Es un mal servicio  negativa
7        Es un producto inútil  negativa
8           No lo recomendaría  negativa
9       Me encantó el producto  positiva
10                 No me gusta  negativa
11               Es fantástico  positiva
12                  Me encanta  positiva
13         Es un buen producto  positiva
14                Es muy bueno  positiva
15          Es un mal servicio  negativa
16    Es una pérdida de dinero  negativa
17         No es nada especial   neutral
18       Es un producto normal   neutral
19                     Te odio  negativa
20         Es un buen servicio  positiva
21         Es un gran producto  positiva
22      Es un producto decente   neutral
23  Es un produc

In [3]:

generator = pipeline('text-generation', model="gpt2", device=0)

# Lista para almacenar las predicciones
predictions_base = []
predictions_zero_shot = []
predictions_few_shot = []
predictions_cot = []

# Clasificar cada opinión y mostrar resultado
for row in df.itertuples():
    opinion = row.opinion
    label = row.label

    prompt_base = f"Clasifica la siguiente opinión en base a como interpretes la actitud del mensaje, dicha actitud puede ser vista como positiva, negativa o neutral: {opinion}"

    zero_shot_prompt = f"Clasifica la siguiente opinión como positiva, negativa o neutral: {opinion}"


    few_shot_prompt = f"""
    Aquí tienes algunas opiniones clasificadas:

    Opinión: 'Me encanta este producto. Es increíble.' → Sentimiento: positivo
    Opinión: 'Este servicio es terrible, nunca lo recomendaría.' → Sentimiento: negativo
    Opinión: 'El artículo está bien, pero no es excepcional.' → Sentimiento: neutral

    Clasifica esta opinión: {opinion}
    """

    cot_prompt = f"""
    Pensando en la siguiente opinión:

    Opinión: '{opinion}'

    El sentimiento puede ser positivo, negativo o neutral. ¿Qué opinas?
    """


    prompts = {
        "prompt": prompt_base,
        "zero-shot": zero_shot_prompt,
        "few-shot": few_shot_prompt,
        "chain-of-thought": cot_prompt
    }

    # Generar predicción con Zero-shot
    prompt_zero_shot = zero_shot_prompt.format(opinion)
    result_zero_shot = generator(prompt_zero_shot, max_length=100, num_return_sequences=1)
    sentiment_zero_shot = result_zero_shot[0]["generated_text"]
    
    
    # Generar predicción con Few-shot
    prompt_few_shot = few_shot_prompt.format(opinion)
    result_few_shot = generator(prompt_few_shot, max_new_tokens=50, num_return_sequences=1)
    sentiment_few_shot = result_few_shot[0]["generated_text"]

    # Generar predicción con el prompt base
    prompt = prompt_base.format(opinion)
    result_base = generator(prompt, max_length=100, num_return_sequences=1)
    sentiment_base = result_base[0]["generated_text"]
    
    # Generar predicción con Chain-of-thought
    prompt_cot = cot_prompt.format(opinion)
    result_cot = generator(prompt_cot, max_length=100, num_return_sequences=1)
    sentiment_cot = result_cot[0]["generated_text"]
    
    # Asignar sentimientos de acuerdo al texto generado
    def assign_sentiment(response):
        normalized_response = response.lower().strip()

        palabras_positivas = ["positivo", "positiva", "positive"]
        palabras_negativas = ["negativo", "negativa", "negative"]
        palabras_neutras = ["neutral", "neutro", "neutra"]

        if any(word in normalized_response for word in palabras_positivas):
            return "positiva"
        elif any(word in normalized_response for word in palabras_negativas):
            return "negativa"
        elif any(word in normalized_response for word in palabras_neutras):
            return "neutral"
        else:
            return "desconocida"
    
    # Asignar sentimientos a cada predicción
    predictions_base.append(assign_sentiment(sentiment_base))
    predictions_zero_shot.append(assign_sentiment(sentiment_zero_shot))
    predictions_few_shot.append(assign_sentiment(sentiment_few_shot))
    predictions_cot.append(assign_sentiment(sentiment_cot))

    # Imprimir resultados
    print(f"Opinión: {opinion} - Label: {label}\n → Zero-shot Sentimiento: {predictions_zero_shot[-1]}")
    print(f" → Few-shot Sentimiento: {predictions_few_shot[-1]}")
    print(f" → Chain-of-thought Sentimiento: {predictions_cot[-1]}")
    print(f" → Base Sentimiento: Sentimiento: {predictions_base[-1]}\n")

# Añadir las predicciones al DataFrame
df['prompt_base'] = prompt_base
df['prediccion_zero_shot'] = predictions_zero_shot
df['prediccion_few_shot'] = predictions_few_shot
df['prediccion_cot'] = predictions_cot

# Mostrar el DataFrame con los resultados
print(df[['opinion', 'label', 'prompt_base' ,'prediccion_zero_shot', 'prediccion_few_shot', 'prediccion_cot']])

Device set to use cuda:0
Truncation was not explicitly activated but `max_length` is provided a specific value, please use `truncation=True` to explicitly truncate examples to max length. Defaulting to 'longest_first' truncation strategy. If you encode pairs of sequences (GLUE-style) with the tokenizer you can select this strategy more precisely by providing a specific strategy to `truncation`.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


Opinión: Es un gran servicio - Label: positiva
 → Zero-shot Sentimiento: positiva
 → Few-shot Sentimiento: positiva
 → Chain-of-thought Sentimiento: positiva
 → Base Sentimiento: Sentimiento: positiva



Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


Opinión: Es regular - Label: neutral
 → Zero-shot Sentimiento: positiva
 → Few-shot Sentimiento: positiva
 → Chain-of-thought Sentimiento: positiva
 → Base Sentimiento: Sentimiento: positiva



Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
You seem to be using the pipelines sequentially on GPU. In order to maximize efficiency please use a dataset
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


...

Opinión: Es un mal producto - Label: negativa
 → Zero-shot Sentimiento: positiva
 → Few-shot Sentimiento: positiva
 → Chain-of-thought Sentimiento: positiva
 → Base Sentimiento: Sentimiento: positiva



Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


Opinión: Es impresionante - Label: positiva
 → Zero-shot Sentimiento: positiva
 → Few-shot Sentimiento: positiva
 → Chain-of-thought Sentimiento: positiva
 → Base Sentimiento: Sentimiento: positiva



Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


Opinión: Es increíble - Label: positiva
 → Zero-shot Sentimiento: positiva
 → Few-shot Sentimiento: positiva
 → Chain-of-thought Sentimiento: positiva
 → Base Sentimiento: Sentimiento: positiva

                       opinion     label  \
0          Es un gran servicio  positiva   
1                   Es regular   neutral   
2          Es un producto útil  positiva   
3                 Es un fraude  negativa   
4               Me gusta mucho  positiva   
5               Es un desastre  negativa   
6           Es un mal servicio  negativa   
7        Es un producto inútil  negativa   
8           No lo recomendaría  negativa   
9       Me encantó el producto  positiva   
10                 No me gusta  negativa   
11               Es fantástico  positiva   
12                  Me encanta  positiva   
13         Es un buen producto  positiva   
14                Es muy bueno  positiva   
15          Es un mal servicio  negativa   
16    Es una pérdida de dinero  negativa   
17         No

In [4]:
classifier = pipeline('sentiment-analysis', model='nlptown/bert-base-multilingual-uncased-sentiment')

for opinion in df['opinion']:
    sentiment = classifier(opinion)
    print(f"Opinión: {opinion} → Sentimiento: {sentiment[0]['label']}")


Device set to use cuda:0


Opinión: Es un gran servicio → Sentimiento: 5 stars
Opinión: Es regular → Sentimiento: 3 stars
Opinión: Es un producto útil → Sentimiento: 4 stars
Opinión: Es un fraude → Sentimiento: 1 star
Opinión: Me gusta mucho → Sentimiento: 5 stars
Opinión: Es un desastre → Sentimiento: 1 star
Opinión: Es un mal servicio → Sentimiento: 1 star
Opinión: Es un producto inútil → Sentimiento: 1 star
Opinión: No lo recomendaría → Sentimiento: 2 stars
Opinión: Me encantó el producto → Sentimiento: 5 stars
Opinión: No me gusta → Sentimiento: 2 stars
Opinión: Es fantástico → Sentimiento: 5 stars
Opinión: Me encanta → Sentimiento: 5 stars
Opinión: Es un buen producto → Sentimiento: 4 stars
Opinión: Es muy bueno → Sentimiento: 5 stars
Opinión: Es un mal servicio → Sentimiento: 1 star
Opinión: Es una pérdida de dinero → Sentimiento: 1 star
Opinión: No es nada especial → Sentimiento: 2 stars
Opinión: Es un producto normal → Sentimiento: 3 stars
Opinión: Te odio → Sentimiento: 1 star
Opinión: Es un buen servic

In [5]:
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM, pipeline

# Load model and tokenizer
model_id = "google/flan-t5-small"
tokenizer = AutoTokenizer.from_pretrained(model_id)
model = AutoModelForSeq2SeqLM.from_pretrained(model_id)

# Create the pipeline
classifier = pipeline('text2text-generation', model=model, tokenizer=tokenizer)

# Define prompt
opinion = "Es un producto normal"
prompt = f"Classify the sentiment of the following text as positive, negative, or neutral: {opinion}"

# Get prediction
output = classifier(prompt)
print(output[0]['generated_text'])


Device set to use cuda:0


negative


**Análisis de aciertos por prompt**

In [6]:
aciertos_df = pd.DataFrame({
    'prompt_base': predictions_base,
    'zero_shot': predictions_zero_shot,
    'few_shot': predictions_few_shot,
    'chain_of_thought': predictions_cot
})

aciertos = {
    'prompt_base': (aciertos_df['prompt_base'] == df['label']).sum(),
    'zero_shot': (aciertos_df['zero_shot'] == df['label']).sum(),
    'few_shot': (aciertos_df['few_shot'] == df['label']).sum(),
    'chain_of_thought': (aciertos_df['chain_of_thought'] == df['label']).sum()
}

print(f"Aciertos con la prompt base: {aciertos['prompt_base']} - Ratio: {aciertos['prompt_base'] / len(aciertos_df):.2f}")
print(f"Aciertos con la prompt zero shot: {aciertos['zero_shot']} - Ratio: {aciertos['zero_shot'] / len(aciertos_df):.2f}")
print(f"Aciertos con la prompt few shots: {aciertos['few_shot']}- Ratio: {aciertos['few_shot'] / len(aciertos_df):.2f}")
print(f"Aciertos con la prompt chain of thought: {aciertos['chain_of_thought']} - Ratio: {aciertos['chain_of_thought'] / len(aciertos_df):.2f}")

Aciertos con la prompt base: 13 - Ratio: 0.43
Aciertos con la prompt zero shot: 13 - Ratio: 0.43
Aciertos con la prompt few shots: 13- Ratio: 0.43
Aciertos con la prompt chain of thought: 13 - Ratio: 0.43
